In [5]:
import os

chunk_size = 1024 * 1024  # 1 MB
# total_size = 1024 * 1024 * 1024  # 1 GB
total_size = 1024 * 1024 * 16  # 16MB, 1024= 1 GB

with open("./bigfile.bin", "wb") as f:
    for _ in range(total_size // chunk_size):
        f.write(os.urandom(chunk_size))

In [2]:
import asyncio

async def blocking_reader():
    print("Blocking reader: start")
    with open("./bigfile.bin", "rb") as f:
        while chunk := f.read(4096):
            print("Blocking reader: read chunk")
            # NO await → event loop cannot switch tasks
    print("Blocking reader: end")

async def periodic():
    while True:
        print("Periodic: tick")
        await asyncio.sleep(0.5)

async def main():
    task1 = asyncio.create_task(blocking_reader())
    task2 = asyncio.create_task(periodic())
    await asyncio.gather(task1, task2)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
from concurrent.futures import ThreadPoolExecutor
import time

def slow():
    time.sleep(10)
    return "done"

with ThreadPoolExecutor() as ex:
    f = ex.submit(slow)
    time.sleep(1)
    cancelled = f.cancel()
    print("Cancelled:", cancelled)
    print(f)

f

Cancelled: False
<Future at 0x1beb405d7d0 state=running>


<Future at 0x1beb405d7d0 state=finished returned str>

In [4]:
f.result()

'done'

In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def task(n):
    time.sleep(n)
    return f"Finished task taking {n}s"

delays = [3, 1, 2]

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(task, d) for d in delays]

    for future in as_completed(futures):
        print(future.result())

Finished task taking 1s
Finished task taking 2s
Finished task taking 3s


In [6]:
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED
import time

def task(n):
    time.sleep(n)
    return n

with ThreadPoolExecutor() as ex:
    futures = [ex.submit(task, i) for i in [1, 2, 3]]

    done, not_done = wait(futures, return_when=FIRST_COMPLETED)

    print("First completed:", [f.result() for f in done])
    print("Remaining:", len(not_done))

First completed: [1]
Remaining: 2
